In [122]:
from k12libs.utils.nb_easy import k12ai_print
from k12libs.utils.nb_easy import k12ai_get_app_dir

import os
import json
import _jsonnet

import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import display, Javascript, HTML
from traitlets import Unicode, Bool, validate, TraitError
from ipywidgets import (DOMWidget, register, interact, interactive,  interact_manual,
                        IntSlider, IntText, FloatText, Text, BoundedIntText,
                        BoundedFloatText, Box, HBox, VBox, fixed, Dropdown,
                        Layout, Tab, Accordion, ToggleButton, ToggleButtons, Checkbox)

In [123]:
cv_schema_dir = os.path.join(k12ai_get_app_dir('cv'), 'templates', 'schema')
cv_json = _jsonnet.evaluate_file(os.path.join(cv_schema_dir, 'k12cv.jsonnet'))
k12ai_print(cv_json)

{
    "data": {
        "dataset": {
            "name": {
                "cn": "Dataset",
                "en": "Dataset"
            },
            "type": "accordion"
        },
        "name": {
            "cn": "data",
            "en": "data"
        },
        "objs": [
            "dataset",
            "transform"
        ],
        "transform": {
            "name": {
                "cn": "Transform",
                "en": "Transform"
            },
            "objs": [
                "train",
                "val",
                "test"
            ],
            "test": {
                "name": {
                    "cn": "Test",
                    "en": "Test"
                },
                "type": "accordion"
            },
            "train": {
                "aug_trans": {
                    "shuffle_trans_seq": {},
                    "trans_seq": {}
                },
                "data_trans": {},
                "name": {
                    "cn": 

In [124]:
k12cv_conf = json.loads(cv_json)

In [125]:
out = widgets.Output()

def k12widget(method):
    def _widget(self, *args, **kwargs):
        cb, wid = method(self, *args, **kwargs)
        def _on_value_change(change, cb):
            with out:
                clear_output()
                print(change)
            if cb:
                cb(change)
        wid.observe(lambda change, cb = cb: _on_value_change(change, cb), 'value')
        return wid
    return _widget

class K12Context():
    def __init__(self):
        self.wids_map = {}
        self.vlo = Layout(width='100%')
        self.hlo = Layout(flex_flow='row wrap', width='100%')

    @k12widget
    def Bool(self, wid, *args, **kwargs):
        self.wids_map[wid] = Checkbox(*args, **kwargs)
        def _value_change(change):
            print("Checkbox here")
        return _value_change, self.wids_map[wid]
    
    @k12widget
    def Int(self, wid, *args, **kwargs):
        self.wids_map[wid] = BoundedIntText(*args, **kwargs)
        def _value_change(change):
            print("BoundedIntText here")
        return _value_change, self.wids_map[wid]
    
    @k12widget
    def Float(self, wid, *args, **kwargs):
        self.wids_map[wid] = BoundedFloatText(*args, **kwargs)
        def _value_change(change):
            print("BoundedFloatText here")
        return _value_change, self.wids_map[wid]
    
    @k12widget
    def String(self, wid, *args, **kwargs):
        self.wids_map[wid] = Text(*args, **kwargs)
        return None, self.wids_map[wid]
    
    @k12widget
    def IntArray(self, wid, *args, **kwargs):
        self.wids_map[wid] = Text(*args, **kwargs)
        def _value_change(change):
            print("IntArray here")
            print(json.loads(change['new']))
        return _value_change, self.wids_map[wid]
    
    @k12widget
    def StringEnum(self, wid, *args, **kwargs):
        self.wids_map[wid] = Dropdown(*args, **kwargs)
        def _value_change(change):
            print("StringEnum here")
        return _value_change, self.wids_map[wid]
    
    @k12widget
    def BoolTrigger(self, wid, *args, **kwargs):
        self.wids_map[wid] = Checkbox(*args, **kwargs)
        self.wids_map[wid].parent_box = VBox(layout = self.hlo)
        self.wids_map[wid].trigger_box = VBox(layout = self.hlo)
        def _switch_widget(wdg, value):
            if value:
                wdg.parent_box.children = [wdg, wdg.trigger_box]
            else:
                wdg.parent_box.children = [wdg]
        def _value_change(change):
            print("BoolTrigger here")
            wdg = change['owner']
            val = change['new']
            _switch_widget(wdg, val)
        _switch_widget(self.wids_map[wid], self.wids_map[wid].value)
        return _value_change, self.wids_map[wid]
    
    @k12widget
    def StringEnumTrigger(self, wid, *args, **kwargs):
        self.wids_map[wid] = Dropdown(*args, **kwargs)
        self.wids_map[wid].parent_box = VBox(layout = self.hlo)
        self.wids_map[wid].trigger_box = {
         value: VBox(layout = self.hlo) for _, value in self.wids_map[wid].options
        }

        def _switch_widget(wdg, value):
            wdg.parent_box.children = [wdg, wdg.trigger_box[value]]
        def _value_change(change):
            print("StringEnumTrigger here")
            wdg = change['owner']
            val = change['new']
            _switch_widget(wdg, val)
        _switch_widget(self.wids_map[wid], self.wids_map[wid].value)
        return _value_change, self.wids_map[wid]
    
    @k12widget
    def StringEnumArrayTrigger(self, wid, *args, **kwargs):
        pass
        
k12Context = K12Context()

In [126]:
lan = 'en'
box_layout = Layout(display='flex',
                flex_flow='row wrap',
                align_items='stretch',
                justify_content='flex-start',
                width='100%')

vlo = Layout(width='100%')
hlo = Layout(flex_flow='row wrap', width='100%')

_basic_types = ['int', 'float', 'bool', 'string', 'string-enum']

def _parse_config(widget, config):
    __id_ = config.get('_id_', None)
    _name = config.get('name', None)
    _type = config.get('type', None)
    _objs = config.get('objs', None)
    if _type == 'page':
        nxtwdt = Tab(layout=Layout(width='100%'))
        widget.children = list(widget.children) + [nxtwdt]
    elif _type == 'tab':
        widget.set_title(len(widget.children), _name[lan])
        nxtwdt = VBox(layout = box_layout)
        widget.children = list(widget.children) + [nxtwdt] 
    elif _type == 'accordion':
        for wdt in widget.children:
            if isinstance(wdt, Accordion):
                accord = wdt
                break
        else:
            accord = Accordion(layout=Layout(
                display='flex',
                align_items='stretch',
                justify_content='flex-start',
                width='100%'
            ))
            widget.children = list(widget.children) + [accord]
        accord.set_title(len(accord.children), _name[lan])
        basic_box = HBox(layout = hlo)
        compx_box = HBox(layout = vlo)
        nxtwdt = VBox((basic_box, compx_box), layout = box_layout)
        accord.children = list(accord.children) + [nxtwdt]
        if _objs and len(_objs) > 0:
            for key in _objs:
                val = config.get(key)
                typ = val.get('type', None)
                if typ in _basic_types:
                    _parse_config(basic_box, val)
                else:
                    _parse_config(compx_box, val)
        return
    elif _type == 'bool':
        default = config.get('default', False)
        wdg = k12Context.Bool(__id_,
             description = _name[lan],
             value = default,
         )
        widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'int':
        default = config.get('default', 0)
        wdg = k12Context.Int(__id_,
            description = _name[lan],
            value = default,
            min = 0,
            max = 100000,
            step = 1,
        )
        widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'float':  # leaf node
        default = config.get('default', 0.0)
        wdg = k12Context.Float(__id_,
            description = _name[lan],
            value = default,
            min = 0.0,
            max = 100.0,
            step = 0.1,
        )
        widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'string':
        default = config.get('default', 'none')
        wdg = k12Context.String(__id_,
            description = _name[lan],
            value = default,
        )
        widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'int-array':
        default = config.get('default', '[]')
        wdg = k12Context.IntArray(__id_,
            description = _name[lan],
            value = json.dumps(default),
        )
        widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'float-array':
        print('not impl')
        return
    elif _type == 'string-enum': # leaf node
        default = config.get('default', 'None')
        options = []
        if _objs and len(_objs) > 0:
            for obj in _objs:
                options.append((obj['name'][lan], obj['value']))
            wdg = k12Context.StringEnum(__id_,
                options = options,
                value = default,
                description = _name[lan])
            widget.children = list(widget.children) + [wdg]
        return widget
    elif _type == 'bool-trigger':
        default = config.get('default', False)
        if _objs and len(_objs) > 0:
            wdg = k12Context.BoolTrigger(__id_,
                  value = default,
                  description = _name[lan])
            for obj in _objs:
                if obj.get('trigger', None):
                    _parse_config(wdg.trigger_box, obj['trigger'])
            widget.children = list(widget.children) + [wdg.parent_box]
        return widget
    elif _type == 'string-enum-trigger':
        default = config.get('default', 'none')
        options = []
        if _objs and len(_objs) > 0:
            for obj in _objs:
                options.append((obj['name'][lan], obj['value']))
            wdg = k12Context.StringEnumTrigger(__id_,
                options = options,
                value = default,
                description = _name[lan])
            for obj in _objs:
                if obj.get('trigger', None):
                    _parse_config(wdg.trigger_box[obj['value']], obj['trigger'])                    
            widget.children = list(widget.children) + [wdg.parent_box]        
        return widget
    elif _type == 'string-enum-array-trigger':
        default = config.get('default', 'none')
        options = {}
#       if _objs and len(_objs) > 0:
#           for obj in _objs:
#               options.append((obj['name'][lan], obj['value']))
#           wdg = HBox(
#               children = [widgets.ToggleButton(description=)]
#               layout = hlo)
 #       toggle_buttons= {'a':widgets.ToggleButton(description='a', value=False),
 #          'b':widgets.ToggleButton(description='b', value=False),
 #          'c':widgets.ToggleButton(description='c', value=False),
 #          'd':widgets.ToggleButton(description='d', value=False)}
        pass
    elif _type == 'object':
        nxtwdt = widget
    else:
        return 
    
    # parse subobject
    if _objs and len(_objs) > 0:
        for key in _objs:
            val = config.get(key, None)
            if val:
                _parse_config(nxtwdt, val)


page = Box()
_parse_config(page, k12cv_conf)
display(page, out)

not impl


Box(children=(Tab(children=(VBox(children=(Accordion(children=(VBox(children=(HBox(layout=Layout(flex_flow='ro…

Output()

In [127]:
print(widgets.__version__)
widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

7.5.1


FileUpload(value={}, description='Upload')